In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from glob import glob
import tifffile
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import datetime
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from tqdm import tqdm
torch.backends.cudnn.benchmark = True

In [2]:
class EmbryoNucleiDataset(Dataset):
    def __init__(self,
                 root_dir,
                 epoch_size
                ):
        
        # using root_dir, split and mask create a path to files and sort it 
        self.mask_files = sorted(glob(os.path.join(root_dir, 'cropped_masks', '*.tif'))) # load mask files into sorted list
        self.raw_files = sorted(glob(os.path.join(root_dir, 'cropped_rawfiles', '*.tif'))) # load image files into sorted list
        self.epoch_size = epoch_size
    
    def __len__(self):
        #return len(self.raw_files)
        return self.epoch_size

    def __getitem__(self, idx):
        idx = np.random.randint(len(self.raw_files))
        raw_file = self.raw_files[idx] 
        mask_file = self.mask_files[idx] 
        crops_raw = tifffile.imread(raw_file) # load raw to numpy array
        crops_mask = tifffile.imread(mask_file) # load mask to numpy array
        crops_mask = (crops_mask !=0).astype(np.float32)
        crops_raw = ((crops_raw.astype(np.float32))/65535) * crops_mask
        
        # add channel dimensions to comply with pytorch standard (B, C, H, W) 
        crops_raw = np.expand_dims(crops_raw, axis=0)
        crops_mask = np.expand_dims(crops_mask, axis=0)
        
        return crops_raw, crops_mask

## Extracting the latent representation of all objects (N=371,012)
Be sure to match all Autoencoder and dataloader params below to match that used in the previous training (found in Training_notebook.py).

In [3]:
class Autoencoder(torch.nn.Module):
    def __init__(
            self,
            in_channels,
            downsampling_factors,
            fmaps,
            fmul,
            fmaps_bottle = 'default',
            kernel_size=3):

        super(Autoencoder, self).__init__()

        out_channels = in_channels

        encoder = []

        for downsampling_factor in downsampling_factors:

            encoder.append(
                    torch.nn.Conv2d(
                        in_channels,
                        fmaps,
                        kernel_size))
            encoder.append(
                    torch.nn.ReLU(inplace=True))
            encoder.append(
                    torch.nn.Conv2d(
                        fmaps,
                        fmaps,
                        kernel_size))
            encoder.append(
                    torch.nn.ReLU(inplace=True))
            encoder.append(
                    torch.nn.MaxPool2d(downsampling_factor))

            in_channels = fmaps

            fmaps = fmaps * fmul

        if fmaps_bottle == 'default':
            fmaps_bottle = fmaps
        
        encoder.append(
            torch.nn.Conv2d(
                in_channels,
                fmaps_bottle,
                kernel_size))
        encoder.append(
            torch.nn.ReLU(inplace=True))

        self.encoder = torch.nn.Sequential(*encoder)

        decoder = []

        fmaps = in_channels

        decoder.append(
            torch.nn.Conv2d(
                fmaps_bottle,
                fmaps,
                kernel_size))
        decoder.append(
            torch.nn.ReLU(inplace=True))

        for idx, downsampling_factor in enumerate(downsampling_factors[::-1]):

            decoder.append(
                torch.nn.Upsample(
                    scale_factor=downsampling_factor,
                    mode='bilinear'))

            in_channels = fmaps
            
            decoder.append(
                torch.nn.Conv2d(
                    in_channels,
                    fmaps,
                    kernel_size))
            decoder.append(
                torch.nn.ReLU(inplace=True))
            if idx < len(downsampling_factors) - 1:
                fmaps = in_channels // fmul
                decoder.append(
                    torch.nn.Conv2d(
                        in_channels,
                        fmaps,
                        kernel_size))
                decoder.append(
                    torch.nn.ReLU(inplace=True))

            else:
                decoder.append(
                    torch.nn.Conv2d(
                        in_channels,
                        out_channels,
                        kernel_size))

        self.decoder = torch.nn.Sequential(*decoder)

    def forward(self, x):

        enc = self.encoder(x)

        dec = self.decoder(enc)

        return enc, dec

### Be careful while setting these values below. They should correspond to the desired model

In [4]:
assert torch.cuda.is_available()
device = torch.device("cuda")
model_depth = 1
downsampling_factor = 4
downsampling_factors = [downsampling_factor]*model_depth
fmaps = 2
fmul = 2
fmaps_bottle = 'default'
kernel_size = 3


model = Autoencoder(in_channels=1, downsampling_factors=downsampling_factors, fmaps=fmaps,
                    fmul=fmul, fmaps_bottle = fmaps_bottle, kernel_size = kernel_size).to(device)

In [ ]:
# Setting directories
root_dir = '/mnt/efs/shared_data/instance_no_gt/20230830_TIF_cellpose_test/'
model_name = 'models/20230903-023210_LONGTRAININGautoencoder_downsamplingfactors_[4]__fmaps_2__fmul_2__fmapsbottle_default__kernelsize_3__loss_MSE.pt'
# Importing the model
model = Autoencoder(in_channels=1, downsampling_factors=[downsampling_factor]*model_depth, fmaps=fmaps, fmul=fmul, kernel_size = kernel_size).to(device)
state = torch.load(root_dir+model_name)
model.load_state_dict(state, strict=True)
#os.mkdir(os.path.join(root_dir, 'latent_spaces', os.path.basename(model_name[:-3])))

In [6]:
# Getting the file names of raw and mask files
raw_files = sorted(glob(os.path.join(root_dir, 'cropped_rawfiles', '*.tif'))) 
mask_files = sorted(glob(os.path.join(root_dir, 'cropped_masks', '*.tif')))

In [7]:
# create train dataset
dataset = EmbryoNucleiDataset(root_dir,epoch_size=len(raw_files))

# create train dataloader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, pin_memory=True)



In [ ]:
# Looping the pre-trained model over the entire dataset: 

# make sure net is in eval mode so we don't backprop
model.eval()

for idx, (raw, mask) in enumerate(tqdm(dataloader)):
    raw = raw.to(device) 
    mask = mask.to(device)
    latent,_ = model(raw) # key line
    latent = latent[0].cpu().detach().numpy()
    filename = os.path.basename(raw_files[idx][:-4])+'.npy'
    np.save(os.path.join(root_dir, 'latent_spaces', os.path.basename(model_name[:-3]), filename), latent)

## Representing the latent space with UMAP plotting
Question: do learned features cluster in potentially meaningful ways?

What we have: a directory of 371,012 latent representations of datatype float32 and shape (1, 64, 74, 74).

### Testing if things are installed correctly (the Penguin Test)
Previous to this, I had followed instructions to install this in the conda env 06_instance_segmentation (via Jupyter terminal).
See: https://umap-learn.readthedocs.io/en/latest/index.html

We need to install seaborn, umap, pandas, and make sure sklearn is version 1.3.0.


In [ ]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd 
%matplotlib inline

# needed to install umap, seaborn, pandas before running these cells

In [ ]:
penguins = pd.read_csv("https://raw.githubusercontent.com/allisonhorst/palmerpenguins/c19a904462482430170bfe2c718775ddb7dbb885/inst/extdata/penguins.csv")
penguins.head()

penguins = penguins.dropna()
penguins.species.value_counts()

sns.pairplot(penguins.drop("year", axis=1), hue='species');

In [ ]:
# Check sklearn version number! if it is NOT 1.3.0, need to pip uninstall scikit-learn and pip install scikit-learn==1.3.0
import sklearn
print(sklearn.__version__)

In [ ]:
from sklearn.metrics import DistanceMetric # needed to uninstall scikit-learn and  pip install scikit-learn==1.3.0 for this import to work
import umap

In [ ]:
reducer = umap.UMAP()

penguin_data = penguins[
    [
        "bill_length_mm",
        "bill_depth_mm",
        "flipper_length_mm",
        "body_mass_g",
    ]
].values
scaled_penguin_data = StandardScaler().fit_transform(penguin_data)

embedding = reducer.fit_transform(scaled_penguin_data)
embedding.shape

plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in penguins.species.map({"Adelie":0, "Chinstrap":1, "Gentoo":2})])
plt.gca().set_aspect('equal', 'datalim')

plt.title('UMAP projection of the Penguin dataset', fontsize=24);

### UMAP for our latent representations

In [ ]:
import os
from glob import glob

In [ ]:
# Importing test data (n=563)
latent_dir = '/mnt/efs/shared_data/instance_no_gt/20230830_TIF_cellpose_test/latent_spaces/'
latent_files = sorted(glob(os.path.join(latent_dir, '*.tif')))
len(latent_files) # length is 563 as we expect

In [ ]:
# Finding a low dimensional representation of the data.
fit = umap.UMAP()
%time u = fit.fit_transform(data)

In [ ]:
print(latent_files[1])
test_image = tifffile.imread(latent_files[0])
#test_image = test_image[1:]
#test_image.shape # (1, 64, 74, 74)
test_image = np.array(test_image)
test_image.shape
#test_image.dtype # float32

In [ ]:
latents = []

for image_file in latent_files:
    # read into image
    image = tifffile.imread(image_file)
    
    # Flatten each image into a 1D vector and stack them vertically
    reshaped_image = image.flatten()

    latents.append(reshaped_image)
    
latents = np.stack(latents) # 562 times number of features

In [ ]:
fit = umap.UMAP()
u = fit.fit_transform(latents) 

In [ ]:
plt.scatter(u[:,0], u[:,1])
plt.title('UMAP embedding of random colours');